## LAB #5

- Derek Arreaga - 22537
- Mónica Salvatierra

Link del repositorio: https://github.com/alee2602/LAB5-DS



## Importación de Librerías

In [1]:

import os, sys, math, warnings, json, random
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from pandas_datareader import data as pdr

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Semillas para reproducibilidad
SEED = 8
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(tf.__version__)


ModuleNotFoundError: No module named 'pandas_datareader'

## 1. Carga y exploración de datos

In [ ]:
SERIE = "IPN31152N"  # Ice cream & frozen dessert production index (2012=100, mensual, NSA)
START = "1985-01-01" # fecha inicial amplia para historia
END   = None         # hasta hoy

def load_fred_series(serie=SERIE, start=START, end=END):
    try:
        df = pdr.DataReader(serie, "fred", start=start, end=end)
        df = df.rename(columns={serie: "value"})
        df.index.name = "date"
        print(f"Descarga exitosa desde FRED: {serie}  |  filas={len(df)}  rango={df.index.min().date()}..{df.index.max().date()}")
        return df
    except Exception as e:
        print("No se pudo descargar desde FRED. Intentando CSV local...", e)
        # CSV de respaldo en el mismo folder del notebook
        local = f"./{serie}.csv"
        if not os.path.exists(local):
            raise FileNotFoundError(f"No hay internet ni archivo local '{local}'. Coloca un CSV con columnas DATE,VALUE.")
        df = pd.read_csv(local)
        # normaliza columnas
        cols = {c.lower(): c for c in df.columns}
        date_col = cols.get("date", None)
        value_col = cols.get("value", None)
        if date_col is None or value_col is None:

            date_col = "DATE" if "DATE" in df.columns else list(df.columns)[0]
            value_col = "VALUE" if "VALUE" in df.columns else list(df.columns)[1]
        df = df.rename(columns={date_col: "date", value_col: "value"})
        df["date"] = pd.to_datetime(df["date"])
        df = df.set_index("date").sort_index()
        print(f"Carga local desde CSV: {local} | filas={len(df)}  rango={df.index.min().date()}..{df.index.max().date()}")
        return df

df = load_fred_series()
display(df.head(3))
display(df.tail(3))


print(df.describe().T)
print("\nFrecuencia inferida:", pd.infer_freq(df.index))


plt.figure(figsize=(10,4))
plt.plot(df.index, df["value"], lw=1.5)
plt.title("Producción industrial: helados y postres congelados (Índice 2012=100, mensual, NSA)")
plt.xlabel("Fecha"); plt.ylabel("Índice")
plt.grid(True, alpha=0.3)
plt.show()
